In [1]:
import os
from dotenv import load_dotenv

from crewai import Agent, Task, Crew
from crewai import LLM
from crewai_tools import SerperDevTool, ScrapeWebsiteTool

from IPython.display import Markdown, display

In [2]:
load_dotenv()

SERPER_API_KEY = os.getenv('SERPER_API_KEY')
os.environ['SERPER_API_KEY'] = SERPER_API_KEY

In [3]:
serper_tool = SerperDevTool(n_results=5)
# website_tool = ScrapeWebsiteTool()

In [4]:
def getLLM():
    llm = LLM(
        model="ollama/llama3.1:8b",
        base_url="http://localhost:11434"
    )
    
    return llm

In [5]:
researcher = Agent(
    role="Pesquisador de Conteúdo",
    goal="Faça uma analise completa do que for solicitado. "
         "Você é um ótimo pesquisador na internet. ",
    tools = [serper_tool],
    verbose=True,
    backstory=(
        "Seu objetivo é buscar por informações relevantes "
        "relacionadas ao que for pedido pelo usuário. "
    ),
    llm=getLLM()
)

In [6]:
pptx_planer = Agent(
    role="Elaborador dos Slides",
    goal="Faça uma analise completa do que for solicitado. "
         "Você é um ótimo pesquisador na internet. ",
    verbose=True,

    backstory=(
        "Seu objetivo é buscar por informações relevantes "
        "relacionadas ao que for pedido pelo usuário. "
    ),
    llm=getLLM()
)

In [7]:
search_strategy_task = Task(
    description=(
        "Pesquise sobre o seguinte tópico: ({topico}). "
        "Pesquise na internet sobre uma explicação do que é "
        "este tópico, como definição, aspectos importantes e o "
        "que vc achar que seja importante de ser levado em conta. "
        "Relacione SEMPRE o tópico com o seguinte tema ({tema})."
    ),
    expected_output=(
        "Um texto elaborado sobre o tópico solicitado, de "
        "forma clara e bem descritiva, que facilite a "
        "compreensão de quem for ler pela primeira vez sobre "
        "({topico}). Não esqueça de colocar a bibliografia"
    ),
    output_file="./output/stackholders_resumo.md",
    agent=researcher,
    llm=getLLM()
)

In [8]:
pptx_strategy_task = Task(
    description=(
        "Ao receber o que foi pesquisado na tarefa anterior, "
        "elabore um planejamento de como colocá-lo em slides, "
        "ou seja, o conteúdo que cada slide deve conter. "
        "Faça de uma maneira a interessar quem for ver os slides "
        "que atraia a atenção do público, não deve conter muito "
        "texto, mas ao mesmo tempo tem que ser algo informativo."
    ),
    expected_output=(
        "Planejamento de ({quantidade}) slides para uma apresentação atrativa "
        "e conteúdo de cada slide, de cada bullet point, e um breve resumo de "
        "como explicar/abordar o público sobre o que está sendo falado em cada slide."
    ),
    output_file="./output/stackholders_slides.md",
    context=[search_strategy_task],
    agent=pptx_planer,
    llm=getLLM()
)

In [9]:
presentation_crew = Crew(
    agents=[researcher,
            pptx_planer],

    tasks=[search_strategy_task,
           pptx_strategy_task],

    verbose=True,
    language="Portuguese"
)

In [10]:
presentation_input = {
    'topico': 'StakeHolders',
    'tema': 'Engenharia da Computacao e Project Management Body of Knowledge (PMBOK)',
    'quantidade': '2'
}

In [ ]:
result = presentation_crew.kickoff(inputs=presentation_input)

In [ ]:
display(Markdown('./output/stackholders_resumo.md'))

In [ ]:
display(Markdown('./output/stackholders_slides.md'))